In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
%load_ext autoreload
%autoreload 2
import lilac as ll

ds = ll.get_dataset('local', 'hncomments-1m')

ll.data.clustering.compute_titles(ds, 'text__cluster')
import lilac as ll
from lilac.batch_utils import flatten_path_iter
from lilac.schema import VALUE_KEY, normalize_path

namespace = 'local'
dataset = 'capybara'
ds = ll.get_dataset(namespace, dataset)
input = 'conversation.*.input'
input_path = normalize_path(input)
text_column = '_text'
cluster_column = '_cluster'
ds.cluster(input_path, output_path=cluster_column, remote=True)


def flatten_input(item) -> str:
  texts = flatten_path_iter(item, input_path)
  # Filter out Nones
  texts = (t for t in texts if t)
  # Deal with enriched items.
  texts = (t[VALUE_KEY] if VALUE_KEY in t else t for t in texts)
  return '\n'.join(texts)


ds.map(flatten_input, output_path=text_column)

In [25]:
import random

groups: dict[int, list[tuple[str, float]]] = {}
for row in ds.select_rows([text_column, cluster_column]):
  cluster_id = row[cluster_column]['cluster_id']
  membership_prob = row[cluster_column]['cluster_membership_prob']
  groups.setdefault(cluster_id, []).append((row[text_column], membership_prob))

# Sort by descending membership score.
for cluster_id, group in groups.items():
  # Remove any duplicate texts in the group.
  group = list(set(group))
  # Shuffle the group to avoid biasing the topic function.
  random.shuffle(group)
  group.sort(key=lambda text_score: text_score[1], reverse=True)
  groups[cluster_id] = group

In [26]:
items = [
  {'cluster_id': cluster_id, 'texts': [{'text': text, 'prob': prob} for text, prob in g]}
  for cluster_id, g in groups.items()
]
ds = ll.from_dicts(namespace, f'{dataset}_clusters', items, overwrite=True)

100%|██████████| 417/417 [00:00<00:00, 6296.60it/s]

Dataset "capybara_clusters" written to ./data/datasets/local/capybara_clusters


In [30]:
print(ds.manifest().data_schema)

cluster_id: int64
texts: list(
  text: string
  prob: float64)


In [2]:
import lilac as ll
from lilac.data.clustering import summarize_request

namespace = 'local'
dataset = 'capybara'
ds = ll.get_dataset(namespace, f'{dataset}_clusters')


def title(item):
  ranked_docs = [(x['text'], x['prob']) for x in item['texts']]
  title = summarize_request(ranked_docs)
  return title


ds.map(title, num_jobs=64, output_path='title_v6', overwrite=True)

[local/capybara_clusters][64 shards] map "title" to "('title_v6',)": 100%|██████████| 417/417 [00:11<00:00, 35.69it/s]


Wrote map output to ./data/datasets/local/capybara_clusters


In [3]:
%autoreload 2
import lilac as ll

ll.set_project_dir('./data')
ds = ll.get_dataset('local', 'capybara')
ll.data.clustering.cluster_impl(ds, 'conversation.*.input', output_path='clusters', remote=True)

[local/capybara][1 shards] map "extract_text" to "('clusters',)": 100%|██████████| 16006/16006 [00:00<00:00, 27559.16it/s]


Wrote map output to clusters-00000-of-00001.parquet


[local/capybara][1 shards] map "compute_clusters" to "('clusters',)": 100%|██████████| 16006/16006 [00:51<00:00, 307.84it/s]


Wrote map output to clusters-00000-of-00001.parquet


[local/capybara][1 shards] map "compute_cluster_titles" to "('clusters',)": 100%|██████████| 16006/16006 [00:31<00:00, 511.24it/s]


Wrote map output to clusters-00000-of-00001.parquet


[local/capybara][1 shards] map "compute_category_clusters" to "('clusters',)": 100%|██████████| 16006/16006 [00:14<00:00, 1091.18it/s]


Wrote map output to clusters-00000-of-00001.parquet


[local/capybara][1 shards] map "compute_category_titles" to "('clusters',)": 100%|██████████| 16006/16006 [00:04<00:00, 3420.32it/s]


Wrote map output to clusters-00000-of-00001.parquet


In [4]:
%autoreload 2
import lilac as ll

ds = ll.get_dataset('local', 'hncomments-1m')
ll.data.clustering.cluster_impl(ds, 'text', remote=True, recompute_titles=True)

[local/hncomments-1m][1 shards] map "extract_text" to "('text__cluster',)": 100%|██████████| 1000000/1000000 [00:28<00:00, 34908.13it/s]


Wrote map output to text__cluster-00000-of-00001.parquet


[local/hncomments-1m][1 shards] map "compute_cluster_titles" to "('text__cluster',)": 100%|██████████| 1000000/1000000 [16:07<00:00, 1033.65it/s]


Wrote map output to text__cluster-00000-of-00001.parquet


[local/hncomments-1m][1 shards] map "compute_category_clusters" to "('text__cluster',)": 100%|██████████| 1000000/1000000 [01:05<00:00, 15177.23it/s]


Wrote map output to text__cluster-00000-of-00001.parquet


[local/hncomments-1m][1 shards] map "compute_category_titles" to "('text__cluster',)": 100%|██████████| 1000000/1000000 [00:31<00:00, 31462.51it/s]


Wrote map output to text__cluster-00000-of-00001.parquet


In [2]:
%autoreload 2
import lilac as ll

ds = ll.get_dataset('local', 'OpenHermes-2.5')
ll.data.clustering.cluster_impl(ds, 'prompt', remote=True)

/Users/dsmilkov/code/lilac/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[local/OpenHermes-2.5][1 shards] map "extract_text" to "('prompt__cluster',)": 100%|██████████| 1001551/1001551 [00:30<00:00, 32375.49it/s]


Wrote map output to prompt__cluster-00000-of-00001.parquet


[local/OpenHermes-2.5][1 shards] map "compute_clusters" to "('prompt__cluster',)": 100%|██████████| 1001551/1001551 [15:47<00:00, 1056.65it/s]


Wrote map output to prompt__cluster-00000-of-00001.parquet


[local/OpenHermes-2.5][1 shards] map "compute_cluster_titles" to "('prompt__cluster',)": 100%|██████████| 1001551/1001551 [13:18<00:00, 1254.06it/s]


Wrote map output to prompt__cluster-00000-of-00001.parquet


[local/OpenHermes-2.5][1 shards] map "compute_category_clusters" to "('prompt__cluster',)": 100%|██████████| 1001551/1001551 [01:17<00:00, 13002.68it/s]


Wrote map output to prompt__cluster-00000-of-00001.parquet


[local/OpenHermes-2.5][1 shards] map "compute_category_titles" to "('prompt__cluster',)": 100%|██████████| 1001551/1001551 [00:19<00:00, 51214.62it/s]


Wrote map output to prompt__cluster-00000-of-00001.parquet


In [1]:
%load_ext autoreload
%autoreload 2

import lilac as ll

ll.set_project_dir('./data')
ds = ll.get_dataset('local', 'openorca')
# ds.cluster('question', remote=True)

/Users/dsmilkov/code/lilac/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Recomputing table+index for openorca... took 21.557s.
